In [1]:
import wfdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as Split
from sklearn.preprocessing import normalize
from wfdb import processing

In [2]:
FREQUENCY = 1300
WINDOW_SIZE = 360
PADDING = 360
%matplotlib tk

In [3]:
def create_set(records):
    print(FREQUENCY)
    signals = np.empty(shape=(0,WINDOW_SIZE+PADDING,1))
    df = pd.read_csv(records)
    sig = df['Column4'].to_numpy()
#     plt.plot(sig)
    print(sig.shape)
    print(df.head())
#     sig = sig[::round(FREQUENCY/360)]
    time = pd.timedelta_range(0, periods=len(sig), freq=f"{(1 / FREQUENCY):.8f}S")
    df = pd.DataFrame(index=time, data={'signal': sig})
    dft = df.resample(f'{1 / 360:.8f}S').sum()  # resample frequency to 360hz
    sig = dft['signal'].values
    print(sig.shape)
    print(df.head())
    qrs_inds = find_peaks(sig, 360)
    df = df.resample(f'{1 / 360:.8f}S').mean()  # resample frequency to 360hz
    sig = df['signal'].values
#     plt.plot(sig)
#     plt.show()
    print(qrs_inds.size)
    for location in qrs_inds:
        half_window_with_padding = int((WINDOW_SIZE + PADDING) / 2)
        if location >= half_window_with_padding and location <= len(sig)-half_window_with_padding:
            signal = sig[(location-half_window_with_padding):(location+half_window_with_padding)].reshape(1, WINDOW_SIZE+PADDING, 1)
            signals = np.append(signals, signal, axis=0)
    print(signals.shape)
    return signals

In [4]:
def find_peaks(sig, fs):
	xqrs = processing.XQRS(sig=sig, fs=fs)
	qrs_inds = processing.xqrs_detect(sig=sig, fs=fs)
	xqrs.detect()
# 	wfdb.plot_items(signal=sig, ann_samp=[qrs_inds])
	return qrs_inds

In [6]:
# FREQUENCY = 360
# signals = create_set('Kemal 360 Hz ECG DATA +5-5HZ.csv')
# FREQUENCY = 500
# signals = create_set('ECG KEMAL 500HZ.csv')
FREQUENCY = 1300
signals = create_set('Kemal ECG 2 1300HZ.csv')

1300
(81108,)
  Column1  Column2 Column3  Column4
0    TIME     1.52     ECG      409
1    TIME     2.17     ECG      410
2    TIME     2.82     ECG      406
3    TIME     3.48     ECG      406
4    TIME     4.14     ECG      401
(22461,)
                           signal
0 days 00:00:00               409
0 days 00:00:00.000769230     410
0 days 00:00:00.001538460     406
0 days 00:00:00.002307690     406
0 days 00:00:00.003076920     401
Learning initial signal parameters...
Found 8 beats during learning. Initializing using learned parameters
Running QRS detection...
QRS detection complete.
Learning initial signal parameters...
Found 8 beats during learning. Initializing using learned parameters
Running QRS detection...
QRS detection complete.
95
(91, 720, 1)


In [ ]:
print(signals.shape)
plt.plot(signals[1].flatten()[WINDOW_SIZE-int(WINDOW_SIZE/2):WINDOW_SIZE+int(WINDOW_SIZE/2)])
plt.show()

In [ ]:
def create_dataframe(signals):
    dfs = pd.DataFrame(signals.reshape(signals.shape[0], signals.shape[1]), index=np.arange(signals.shape[0]))
#     df = pd.concat([dfa, dfs], axis=1)
    return dfs

In [ ]:
df = create_dataframe(signals)
df.head

In [ ]:
df.to_csv('collab_1300hz_data.csv')